In [4]:
import yfinance as yf
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

In [5]:
stock_symbol_list = [
    "KO", "PEP", "MDLZ", "TSN", "ADM",  # Agroalimentaire
    "MET", "PRU", "AFL", "AIG", "ALL",  # Assurance
    "DIS", "NFLX", "CMCSA", "LYV", "FOXA",  # Divertissement
    "WMT", "AMZN", "TGT", "COST", "HD",  # Distribution
    "XOM", "CVX",  "BP",  # Énergie
    "AAPL", "MSFT", "GOOGL", "NVDA", "TSLA",  # Technologie
    "PFE", "MRK", "GSK", "NVS", "ABBV",  # Pharmaceutique
    "MMM", "HON", "GE", "CAT", "BA",  # Industriel
    "NKE", "LULU", "PVH", "GPS", "FL",  # Mode
    "NKE", "UAA", "ADDYY", "DIS", "DKNG",  # Sport
    "T", "VZ", "TMUS",   # Télécommunication
    "CRM", "BABA", "SQ", "PYPL", "MELI"  # Businnes
]


In [6]:
for L in stock_data_list:
    
    #Compare avec la veille
    L['Target'] = np.log(L['Close'] / L['Close'].shift(-1)) * 100 #Rendement futur
    
    # Calcul des quantiles
    quantile_20 = L['Target'].quantile(0.2)
    quantile_80 = L['Target'].quantile(0.8)


    
    conditions = [
        (L['Target'] > quantile_80 ),
        (L['Target'] <  quantile_20 ),
    ]
    
    # Définition des valeurs correspondantes
    values = [1, -1]
    
    L['Target'] = np.select(conditions, values, default=0)
    

NameError: name 'stock_data_list' is not defined

In [ ]:
combined_data = pd.concat(stock_data_list)
print(combined_data)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.utils import resample

In [ ]:
from sklearn.ensemble import RandomForestClassifier#peut détecter les non linearités entre les parametres
#random traine un groupe d'arbre de décision individuels avce des paramétres randoms puis faire la moyenne de ces resulats 
model=RandomForestClassifier(n_estimators=100,min_samples_split=100,random_state=1)
#n estimators le nombre de decisions individuels que nous voulons trainer si ce nombre augement la precision est plus elevé
#min samples split pour definir la repartition minimale de l'echantillon higher the less accurate
# random state signifie si on exécute le modéle deux fois les nombres généres seront dans une séquence prévisible si on tourne le code deux fois on aura les memes resulats
# separer nos donnes en deux données d'entrainement et données de test
train = combined_data.iloc[:-100].copy()
test = combined_data.iloc[-100:].copy()

# Handle class imbalance with resampling
train_majority = train[train.Target == 0]
train_minority_pos = train[train.Target == 1]
train_minority_neg = train[train.Target == -1]

train_minority_pos_upsampled = resample(train_minority_pos,
                                        replace=True,     # sample with replacement
                                        n_samples=len(train_majority),    # to match majority class
                                        random_state=123) # reproducible results

train_minority_neg_upsampled = resample(train_minority_neg,
                                        replace=True,     # sample with replacement
                                        n_samples=len(train_majority),    # to match majority class
                                        random_state=123) # reproducible results

train_upsampled = pd.concat([train_majority, train_minority_pos_upsampled, train_minority_neg_upsampled])

# Normalize the data
scaler = StandardScaler()
train_upsampled.loc[:, predictors] = scaler.fit_transform(train_upsampled[predictors])
test.loc[:, predictors] = scaler.transform(test[predictors])

# Define and train the model
model = RandomForestClassifier(n_estimators=100, min_samples_split=100, random_state=1)
model.fit(train_upsampled[predictors], train_upsampled["Target"])

# Predict results on the test data
preds = model.predict(test[predictors])

# Calculate model precision
precision = precision_score(test["Target"], preds, average='macro', zero_division=0)
print(f"Précision du modèle : {precision}")

# Convert predictions to a Pandas Series for easy manipulation
preds = pd.Series(preds, index=test.index)

# Combine predictions and actual values for comparison
combined = pd.concat([test["Target"], preds], axis=1)
combined.columns = ['Actual', 'Predicted']

# Display results
plt.figure(figsize=(14, 7))
plt.plot(combined.index, combined['Actual'], label='Actual', color='blue')
plt.plot(combined.index, combined['Predicted'], label='Predicted', color='orange', linestyle='--')
plt.xlabel("Date")
plt.ylabel("Value")
plt.title("Actual vs Predicted Values")
plt.legend()
plt.show()